## Подготовка данных для задания

In [1]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b0e6d83fcd04bcf734f17b0a9bebcfe47c5d5024f96d5fe26360aef0d68931a6
  Stored in directory: c:\users\орлан\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [12]:
import re

import pandas as pd
import pymorphy2

text_df = pd.read_csv("content_description.csv", sep='\t')
#text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)

# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False)
word_count_df

,doc_id,word,dummy
0,0,1895,1
1,0,а,1
2,0,абсолютно,1
3,0,англия,1
4,0,безобразный,1
...,...,...,...
834,7,экономический,1
835,7,экран,1
836,7,эльгорт,1
837,7,энсесть,1


## 12.3  Практическая работа

12.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [2]:
doc_frequency = word_count_df.groupby('word')['doc_id'].count().sort_values(ascending=False).reset_index()
doc_frequency['doc_frequency'] = round((doc_frequency['doc_id'] / (word_count_df['doc_id'].max() + 1)) * 100)
#doc_frequency

# выведите на экран топ-10 слов
doc_frequency = pd.DataFrame(doc_frequency.loc[:, ['word', 'doc_frequency']])
doc_frequency.head(10)

# -- ВАШ КОД ЗДЕСЬ --
morph = pymorphy2.MorphAnalyzer()
pre = ['INTJ', 'PRCL', 'CONJ', 'PREP']
words = []
for word in doc_frequency['word']:
    w = morph.parse(word)[0].tag.POS
    #print(w)
    if w not in pre:
        words.append(word)
#print(words)
doc_frequency[doc_frequency['word'].isin(words)].head()

,word,doc_frequency
4,он,75.0
5,свой,75.0
7,онлайн,75.0
9,год,62.0
10,смотреть,62.0


12.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [3]:
raw_text = str(text_df.loc[[2], 'description'])
# запишите необходимое регулярное выражение в переменную reg_expr 
reg_expr = r'[А-Я][а-я]+\s[А-Я][а-я]+'

# скомпилированное выражение добавьте в обьёект reg_expr_compiled
reg_expr_compiled = re.compile(reg_expr)

# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Миловидный Давид


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`, сохранить в переменнуб `genres_df`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [35]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}
#genres_df = pd.DataFrame()
nested_genres = [[(i, j) for j in genre_dict[i]] for i in genre_dict]
#nested_genres
flatten_genres = list(itertools.chain(*nested_genres))
#flatten_genres
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'word'])
#genres_df
genres_df = genres_df[['word', 'genre']]
#genres_df
merged_df = word_count_df.merge(genres_df, on = 'word', how = 'left')
merged_df_doc = pd.DataFrame(merged_df.groupby('doc_id')['genre'].unique().reset_index())
merged_df_doc.head()

,doc_id,genre
0,0,"[nan, детектив, триллер]"
1,1,"[nan, сказка]"
2,2,"[nan, мелодрама]"
3,3,"[nan, комедия]"
4,4,"[nan, комедия]"
